# Intialising all the library and package

In [ ]:
# -*- coding: utf-8 -*-
import os
import cv2
import h5py
import numpy


# Reading the HR and LR mage for preprocessing for training

In [41]:

DATA_PATH_HR = "./Train/HR/"
DATA_PATH_LR = "./Train/LR/"
TEST_PATH = "./Test/Set14/"
train_img_num = 100
test_img_num = 14
Random_Crop = 400
Patch_size = 32
label_size = 20
conv_side = 6
scale = 2

# Pre Processing the data, With upscaling the LR image and making small patches 

In [42]:
def prepare_training_data():
    names1 = os.listdir(DATA_PATH_HR)
    names2 = os.listdir(DATA_PATH_LR)
    
    names1 = sorted(names1)
    names2 = sorted(names2)
    nums1 = names1.__len__()
    nums2 = names2.__len__()

    data = numpy.zeros((train_img_num * Random_Crop, 1, Patch_size, Patch_size), dtype=numpy.double)
    label = numpy.zeros((train_img_num * Random_Crop, 1, label_size, label_size), dtype=numpy.double)

    if nums2 < train_img_num:
        print("training img is not enough")

    for i in range(train_img_num):
        name1 = DATA_PATH_HR + names1[i]
        name2 = DATA_PATH_LR + names2[i]
        hr_img = cv2.imread(name1, cv2.IMREAD_COLOR)
        lr_img = cv2.imread(name2, cv2.IMREAD_COLOR)
        shape1 = hr_img.shape
        shape2 = lr_img.shape

        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:, :, 0]
        
        lr_img = cv2.cvtColor(lr_img, cv2.COLOR_BGR2YCrCb)
        lr_img = lr_img[:, :, 0]

        # two resize operation to produce training data and labels
        #lr_img1 = cv2.resize(hr_img, (int(shape1[1] / scale), int(shape1[0] / scale)))
        #print(lr_img.shape)
        lr_img = cv2.resize(lr_img, (shape1[1], shape1[0]))

        #print(lr_img.shape)
        # produce Random_Crop random coordinate to crop training img
        Points_x = numpy.random.randint(0, min(shape1[0], shape1[1]) - Patch_size, Random_Crop)
        Points_y = numpy.random.randint(0, min(shape1[0], shape1[1]) - Patch_size, Random_Crop)

        for j in range(Random_Crop):
            lr_patch = lr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]
            hr_patch = hr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]

            lr_patch = lr_patch.astype(float) / 255.
            hr_patch = hr_patch.astype(float) / 255.

            data[i * Random_Crop + j, 0, :, :] = lr_patch
            label[i * Random_Crop + j, 0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]
            #cv2.imshow("lr", lr_patch)
            #cv2.imshow("hr", hr_patch)
            #cv2.waitKey(0)
    return data, label

# Write the cropped data into h5 file 

In [46]:
def write_hdf5(data, labels, output_filename):
    """
    This function is used to save image data and its label(s) to hdf5 file.
    output_file.h5,contain data and label
    """

    x = data.astype(numpy.float32)
    y = labels.astype(numpy.float32)

    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
        # h.create_dataset()

In [48]:


if __name__ == "__main__":
    data, label = prepare_training_data()
    write_hdf5(data, label, "train.h5")
    # _, _a = read_training_data("train.h5")